<a href="https://colab.research.google.com/github/victorbahlangene/fastai-fastbook22/blob/main/U_S_Patent_Phrase_to_Phrase_Matching_transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Getting started with NLP for absolute beginners

## U.S. Patent Phrase to Phrase Matching

...

## Data

### Importing data from kaggle

In [ ]:
import os
iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')

In [ ]:
# to use kaggle api #
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# kaggle API creds #
creds = '{"username":"victorbahlangene","key":"b744cee991dda6276ec0b54d95413272"}'

In [ ]:
# Then execute this cell (this only needs to be run once) #
from pathlib import Path

cred_path = Path('~/.kaggle/kaggle.json').expanduser()
if not cred_path.exists():
    cred_path.parent.mkdir(exist_ok=True)
    cred_path.write_text(creds)
    cred_path.chmod(0o600)

In [ ]:
# path where datasets will be located #
path = Path('us-patent-phrase-to-phrase-matching')

**Actual download of dataset**

In [ ]:
#  Kaggle API to download the dataset to that path, and extract it #
if not iskaggle and not path.exists():
    import zipfile,kaggle
    kaggle.api.competition_download_cli(str(path))
    zipfile.ZipFile(f'{path}.zip').extractall(path)

100%|██████████| 682k/682k [00:00<00:00, 63.1MB/s]

**Installing huggingface transformers and datasets**

In [ ]:
# huggingface transformers #
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 32.6 MB/s 
     |████████████████████████████████| 596 kB 43.2 MB/s 
     |████████████████████████████████| 101 kB 12.6 MB/s 
     |████████████████████████████████| 6.6 MB 51.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
# installing hugging face datasets #
! pip install -q datasets

     |████████████████████████████████| 365 kB 35.6 MB/s 
     |████████████████████████████████| 212 kB 74.7 MB/s 
     |████████████████████████████████| 141 kB 30.3 MB/s 
     |████████████████████████████████| 115 kB 26.9 MB/s 
     |████████████████████████████████| 127 kB 525 kB/s 


### Data exploration

In [ ]:
!ls {path}

sample_submission.csv  test.csv  train.csv


Open data using pandas Lib

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv(path/"train.csv")
df.head()

,id,anchor,target,context,score
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75
2,36d72442aefd8232,abatement,active catalyst,A47,0.25
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00


Exploritory data analysis

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36473 entries, 0 to 36472
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   id       36473 non-null  object 
 1   anchor   36473 non-null  object 
 2   target   36473 non-null  object 
 3   context  36473 non-null  object 
 4   score    36473 non-null  float64
dtypes: float64(1), object(4)
memory usage: 1.4+ MB


In [ ]:
df.describe()

,score
count,36473.000000
mean,0.362062
std,0.258335
min,0.000000
25%,0.250000
50%,0.250000
75%,0.500000
max,1.000000


In [ ]:
# show for categorical cols #
df.describe(include="object")

,id,anchor,target,context
count,36473,36473,36473,36473
unique,36473,733,29340,106
top,37d61fd2272659b1,component composite coating,composition,H01
freq,1,152,24,2186


**Combining the cols by concatinating them**

In [ ]:
df.head()

,id,anchor,target,context,score
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75
2,36d72442aefd8232,abatement,active catalyst,A47,0.25
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00


In [ ]:
'TEXT1: ' + df.context + '; TEXT2: ' + df.target + '; ANC1: ' + df.anchor

0        TEXT1: A47; TEXT2: abatement of pollution; ANC...
1        TEXT1: A47; TEXT2: act of abating; ANC1: abate...
2        TEXT1: A47; TEXT2: active catalyst; ANC1: abat...
3        TEXT1: A47; TEXT2: eliminating process; ANC1: ...
4        TEXT1: A47; TEXT2: forest region; ANC1: abatement
                               ...                        
36468    TEXT1: B44; TEXT2: wooden article; ANC1: wood ...
36469    TEXT1: B44; TEXT2: wooden box; ANC1: wood article
36470    TEXT1: B44; TEXT2: wooden handle; ANC1: wood a...
36471    TEXT1: B44; TEXT2: wooden material; ANC1: wood...
36472    TEXT1: B44; TEXT2: wooden substrate; ANC1: woo...
Length: 36473, dtype: object

In [ ]:
df["input"] = 'TEXT1: ' + df.context + '; TEXT2: ' + df.target + '; ANC1: ' + df.anchor

In [ ]:
df.head()

,id,anchor,target,context,score,input
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50,TEXT1: A47; TEXT2: abatement of pollution; ANC...
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75,TEXT1: A47; TEXT2: act of abating; ANC1: abate...
2,36d72442aefd8232,abatement,active catalyst,A47,0.25,TEXT1: A47; TEXT2: active catalyst; ANC1: abat...
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50,TEXT1: A47; TEXT2: eliminating process; ANC1: ...
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00,TEXT1: A47; TEXT2: forest region; ANC1: abatement


In [ ]:
# view first input #
df.loc[0,"input"]

'TEXT1: A47; TEXT2: abatement of pollution; ANC1: abatement'

## Tokenization

**Creating a huggingface dataset**

`Dataset` is for the actual data. `DatasetDict` is for the metrics.

In [ ]:
from datasets import Dataset,DatasetDict

In [ ]:
ds = Dataset.from_pandas(df)

In [ ]:
ds

Dataset({
    features: ['id', 'anchor', 'target', 'context', 'score', 'input'],
    num_rows: 36473
})

Tekenization depends on the model we select. The tutorial is using `microsoft/deberta-v3-small`
__(replace "small" with "large" for a slower but more accurate model, once you've finished exploring)__

for this version we are using `distilbert-base-uncased`

In [ ]:
# hugging face model #
model_nm = 'distilbert-base-uncased'

`AutoTokenizer` will create a tokenizer appropriate for a given model:

In [ ]:
# import AutoModelForSequenceClassification and AutoTokenizer for tokenization and model building #
from transformers import AutoModelForSequenceClassification,AutoTokenizer

In [ ]:
# tokenizer from model #
tokz = AutoTokenizer.from_pretrained(model_nm)

In [ ]:
# view tokenizer #
tokz

PreTrainedTokenizerFast(name_or_path='distilbert-base-uncased', vocab_size=30522, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

Test tokenizer

In [ ]:
tokz.tokenize("G'day folks, I'm Jeremy from fast.ai!")

['g',
 "'",
 'day',
 'folks',
 ',',
 'i',
 "'",
 'm',
 'jeremy',
 'from',
 'fast',
 '.',
 'ai',
 '!']

In [ ]:
tokz.tokenize("A platypus is an ornithorhynchus anatinus.")

['a',
 'pl',
 '##at',
 '##yp',
 '##us',
 'is',
 'an',
 'or',
 '##ni',
 '##thor',
 '##hy',
 '##nch',
 '##us',
 'ana',
 '##tin',
 '##us',
 '.']

**Here's a simple function which tokenizes our inputs:**

made change here from:

```python
def tok_func(x):
  return tokz(x["input"])
```
to:

```python
def tok_func(x):
  return tokz(x["input"], truncation=True, padding="max_length")
```

In [ ]:
def tok_func(x):
  return tokz(x["input"], truncation=True, padding=True)

**To run this quickly in parallel on every row in our dataset, use map:**

In [ ]:
tok_ds = ds.map(tok_func, batched=True)

  0%|          | 0/37 [00:00<?, ?ba/s]

In [ ]:
tok_ds

Dataset({
    features: ['id', 'anchor', 'target', 'context', 'score', 'input', 'input_ids', 'attention_mask'],
    num_rows: 36473
})

In [ ]:
# view input and input_ids of the first row #
row = tok_ds[0]
row["input"], row["input_ids"]

('TEXT1: A47; TEXT2: abatement of pollution; ANC1: abatement',
 [101,
  3793,
  2487,
  1024,
  1037,
  22610,
  1025,
  3793,
  2475,
  1024,
  19557,
  18532,
  4765,
  1997,
  10796,
  1025,
  2019,
  2278,
  2487,
  1024,
  19557,
  18532,
  4765,
  102,
  0,
  0,
  0,
  0,
  0,
  0,
  0])

**View tokenizer vocab**

In [ ]:
# for word 'pollution' #
tokz.vocab["pollution"]

10796

Finally, we need to prepare our labels. Transformers always assumes that your labels has the column name labels, but in our dataset it's currently score. Therefore, we need to rename it:

In [ ]:
tok_ds

Dataset({
    features: ['id', 'anchor', 'target', 'context', 'score', 'input', 'input_ids', 'attention_mask'],
    num_rows: 36473
})

In [ ]:
tok_ds = tok_ds.rename_columns({'score':'labels'})

In [ ]:
tok_ds

Dataset({
    features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'attention_mask'],
    num_rows: 36473
})

## Test and validation sets

### Test set (*view only after model is built from the train and validation sets*)

In [ ]:
# we call it 'eval_df' so its easier to understand  #
eval_df = pd.read_csv(path/'test.csv')
eval_df.describe()

,id,anchor,target,context
count,36,36,36,36
unique,36,34,36,29
top,4112d61851461f60,el display,inorganic photoconductor drum,G02
freq,1,2,1,3


### Validation data set

**In huggingface the validation set is refered to as the test set**

Transformers uses a `DatasetDict` for holding your training and validation sets. To create one that contains 25% of our data for the validation set, and 75% for the training set, use `train_test_split`:

In [ ]:
# seed if so we can control the randomness for model improvement purposes #
dds = tok_ds.train_test_split(0.25, seed=42)
dds

DatasetDict({
    train: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'attention_mask'],
        num_rows: 27354
    })
    test: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'attention_mask'],
        num_rows: 9119
    })
})

**Apply the changes to 'eval_df' we performed to 'ds'**

In [ ]:
# concatinating 'eval_df.context' and 'eval_df.target' and 'eval_df.anchor' #
eval_df['input'] = 'TEXT1: ' + eval_df.context + '; TEXT2: ' + eval_df.target + '; ANC1: ' + eval_df.anchor
# create 'eval_ds' dataset and apply tokkenization to the dataset #
eval_ds = Dataset.from_pandas(eval_df).map(tok_func, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
eval_ds

Dataset({
    features: ['id', 'anchor', 'target', 'context', 'input', 'input_ids', 'attention_mask'],
    num_rows: 36
})

In [ ]:
# first row of 'eval_ds' #
eval_row = eval_ds[0]
eval_row["input"], eval_row["input_ids"]

('TEXT1: G02; TEXT2: inorganic photoconductor drum; ANC1: opc drum',
 [101,
  3793,
  2487,
  1024,
  1043,
  2692,
  2475,
  1025,
  3793,
  2475,
  1024,
  28256,
  6302,
  8663,
  8566,
  16761,
  6943,
  1025,
  2019,
  2278,
  2487,
  1024,
  6728,
  2278,
  6943,
  102,
  0])

## Metrics and correlation

**Pearson correlation coefficient**: This coefficient is usually abbreviated using the single letter `r`. It is the most widely used measure of the degree of relationship between two variables.

r can vary between -1, which means perfect inverse correlation, and +1, which means perfect positive correlation.

**Transformers expects metrics to be returned as a dict, since that way the trainer knows what label to use, so let's create a function to do that:**

In [ ]:
# so as to use 'np.corrcoef' #
import numpy as np

Therefore, we'll create this little function to just return the single number we need given a pair of variables:

In [ ]:
def corr(x,y):
  return np.corrcoef(x,y)[0][1]

In [ ]:
np.concatenate((X[i], y), axis=1)

Transformers expects metrics to be returned as a dict, since that way the trainer knows what label to use, so let's create a function to do that:

In [ ]:
def corr_d(eval_pred):
  return {'pearson': corr(*eval_pred)}

## Training

### Training model

In [ ]:
# for training in transformers #
from transformers import TrainingArguments,Trainer

batch size and training epochs

In [ ]:
bs = 32 # initially 128
epochs = 4

learning rate

In [ ]:
lr = 8e-5

**Training arguments:** we normally only change 3 (lr,bs,epochs) 

In [ ]:
args = TrainingArguments('outputs', 
                         learning_rate=lr, 
                         warmup_ratio=0.1, 
                         lr_scheduler_type='cosine', 
                         fp16=True,
                         evaluation_strategy="epoch",
                         per_device_train_batch_size=bs,
                         per_device_eval_batch_size=bs*2,
                         num_train_epochs=epochs,
                         weight_decay=0.01,
                         report_to='none')

Create a trainer

In [ ]:
# the model we choose 'num_labels=1' because univariate analysis #
model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=1)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'classifi

In [ ]:

trainer = Trainer(model, args, train_dataset=dds['train'], eval_dataset=dds['test'],
                  tokenizer=tokz, compute_metrics=corr_d)

Using cuda_amp half precision backend


# Test

In [ ]:
bs1 = 32
epochs1 = 2
lr1 = 8e-5

In [ ]:
args1 = TrainingArguments('outputs', 
                         learning_rate=lr, 
                         warmup_ratio=0.1, 
                         lr_scheduler_type='cosine', 
                         fp16=True,
                         evaluation_strategy="epoch",
                         per_device_train_batch_size=bs,
                         per_device_eval_batch_size=bs*2,
                         num_train_epochs=epochs,
                         weight_decay=0.01,
                         report_to='none')

PyTorch: setting up devices


In [ ]:
def corr1(x,y):
  return np.corrcoef(x,[y])[0][1]

In [ ]:
# np.concatenate((X[i], y), axis=1)

In [ ]:
def corr_d1(eval_pred):
  return {'pearson': corr(*eval_pred)}

In [ ]:
trainer2 = Trainer(model, args1, train_dataset=dds['train'], eval_dataset=dds['test'],
                  tokenizer=tokz, compute_metrics=corr_d1)

Using cuda_amp half precision backend


In [ ]:
trainer2.train();

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: context, input, anchor, target, id. If context, input, anchor, target, id are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 27354
  Num Epochs = 4
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 3420


Epoch,Training Loss,Validation Loss


Saving model checkpoint to outputs/checkpoint-500
Configuration saved in outputs/checkpoint-500/config.json
Model weights saved in outputs/checkpoint-500/pytorch_model.bin
tokenizer config file saved in outputs/checkpoint-500/tokenizer_config.json
Special tokens file saved in outputs/checkpoint-500/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: context, input, anchor, target, id. If context, input, anchor, target, id are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 9119
  Batch size = 64


ValueError: ignored

# end of test

In [ ]:
trainer1 = Trainer(model, args, train_dataset=dds['train'], eval_dataset=dds['test'],
                  tokenizer=tokz)

Using cuda_amp half precision backend


**Train model**

In [ ]:
trainer1.train();

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: context, input, anchor, target, id. If context, input, anchor, target, id are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 27354
  Num Epochs = 4
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 3420


Epoch,Training Loss,Validation Loss
1,0.024300,0.030377
2,0.021000,0.027355
3,0.012100,0.025520
4,0.008600,0.024804


Saving model checkpoint to outputs/checkpoint-500
Configuration saved in outputs/checkpoint-500/config.json
Model weights saved in outputs/checkpoint-500/pytorch_model.bin
tokenizer config file saved in outputs/checkpoint-500/tokenizer_config.json
Special tokens file saved in outputs/checkpoint-500/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: context, input, anchor, target, id. If context, input, anchor, target, id are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 9119
  Batch size = 64
Saving model checkpoint to outputs/checkpoint-1000
Configuration saved in outputs/checkpoint-1000/config.json
Model weights saved in outputs/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in outputs/checkpoint-1000/tokenizer_config.json
Special tokens file sav

In [ ]:
# trainer.train();

# Predictions

In [ ]:
preds = trainer1.predict(eval_ds).predictions.astype(float)
preds

Look out - some of our predictions are <0, or >1! This once again shows the value of remember to actually look at your data. Let's fix those out-of-bounds predictions:

In [ ]:
preds = np.clip(preds, 0, 1)

In [ ]:
preds

array([[0.37963867],
       [0.59130859],
       [0.43383789],
       [0.23937988],
       [0.        ],
       [0.31713867],
       [0.46972656],
       [0.        ],
       [0.23449707],
       [0.91259766],
       [0.        ],
       [0.29443359],
       [0.68408203],
       [0.78662109],
       [0.67724609],
       [0.4440918 ],
       [0.19189453],
       [0.01922607],
       [0.71435547],
       [0.4375    ],
       [0.48242188],
       [0.22351074],
       [0.24438477],
       [0.24182129],
       [0.46655273],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.42626953],
       [0.35839844],
       [0.        ],
       [0.70800781],
       [0.49267578],
       [0.24438477],
       [0.25292969]])